# Nearly-Gaussian Distributions

In [1]:
import sympy as sp
from symlib.random import ExpVal
from symlib.gaussian import wick_contraction, GaussianIndexedBase, GaussianExpVal
from symlib.utils import wilds, wild_subs, pull_sums_out_front

we have distribution 
$$
p(z) \equiv \frac{e^{-S(z)}}{Z}
$$
where $S(z)$ is action and $Z$ is partition function, we consider model:
$$
S(z) = \frac{1}{2} \sum_{\mu, \nu = 1}^{N} K^{\mu \nu} z_\mu z_\nu 
+ \frac{\epsilon}{4!} \sum_{\mu, \nu, \rho, \lambda = 1}^{N} 
V^{\mu \nu \rho \lambda} z_\mu z_\nu z_\rho z_\lambda
$$
where the second term is a small quartic_action

In [2]:
K = sp.IndexedBase('K')
V = sp.IndexedBase('V')
mu = sp.symbols('mu1:5', integer = True)
rho = sp.symbols('rho1:5', integer = True)
eps = sp.Symbol('epsilon')
z = GaussianIndexedBase('z')
N = sp.Symbol('N', integer = True)

In [3]:
Z0 = sp.Symbol('\sqrt{|2\pi K|}') #sp.sqrt(sp.Abs(2*sp.pi*K), evaluate=False)
Z0 # partition function for quadratic action

\sqrt{|2\pi K|}

In [4]:
quartic_action = - eps * sp.Sum(V[*rho]*z[rho[0]]*z[rho[1]]*z[rho[2]]*z[rho[3]]/24, 
                (rho[0], 1, N),
                (rho[1], 1, N),
                (rho[2], 1, N),
                (rho[3], 1, N),
                )
quartic_action

-epsilon*Sum(z[rho1]*z[rho2]*z[rho3]*z[rho4]*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N))

## Partition Function
$$
Z = \int \left[ \prod_{\mu} dz_\mu \right] e^{-S(z)} \\
= \int \left[ \prod_{\mu} dz_\mu \right] \exp \left( 
    -\frac{1}{2} \sum_{\mu, \nu} K^{\mu \nu} z_\mu z_\nu 
    - \frac{\epsilon}{24} \sum_{\rho_1, \ldots, \rho_4} 
    V^{\rho_1 \rho_2 \rho_3 \rho_4} z_{\rho_1} z_{\rho_2} z_{\rho_3} z_{\rho_4}
\right) \\
= \sqrt{|2 \pi K|} \left\langle \exp \left(
    - \frac{\epsilon}{24} \sum_{\rho_1, \ldots, \rho_4} 
    V^{\rho_1 \rho_2 \rho_3 \rho_4} z_{\rho_1} z_{\rho_2} z_{\rho_3} z_{\rho_4}
\right) \right\rangle_K .
$$
let's work out $Z$:

In [5]:
EK = GaussianExpVal(K)
Z = Z0 * EK(sp.series(sp.exp(quartic_action), x=eps, x0=0, n=2).removeO())
Z

\sqrt{|2\pi K|}*(-epsilon*Sum(⟨z[rho1]*z[rho2]*z[rho3]*z[rho4]⟩_K*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + 1)

In [6]:
Z = wick_contraction(Z)
Z

\sqrt{|2\pi K|}*(-epsilon*Sum((⟨z[rho1]*z[rho2]⟩_K*⟨z[rho3]*z[rho4]⟩_K + ⟨z[rho1]*z[rho3]⟩_K*⟨z[rho2]*z[rho4]⟩_K + ⟨z[rho1]*z[rho4]⟩_K*⟨z[rho2]*z[rho3]⟩_K)*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + 1)

In [7]:
A, B = wilds('A, B')
gaussian_rule = {EK(z[A] * z[B]):K[A, B]}
Z = wild_subs(Z, gaussian_rule)
Z

\sqrt{|2\pi K|}*(-epsilon*Sum((K[rho1, rho2]*K[rho3, rho4] + K[rho1, rho3]*K[rho2, rho4] + K[rho1, rho4]*K[rho2, rho3])*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + 1)

assume full symmetry of $K$ and $V$, so we can sort the index:

In [8]:
Z = pull_sums_out_front(sp.expand(Z))
Z

\sqrt{|2\pi K|} + Sum(-\sqrt{|2\pi K|}*epsilon*K[rho1, rho2]*K[rho3, rho4]*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + Sum(-\sqrt{|2\pi K|}*epsilon*K[rho1, rho3]*K[rho2, rho4]*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + Sum(-\sqrt{|2\pi K|}*epsilon*K[rho1, rho4]*K[rho2, rho3]*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N))

In [9]:
def sort_indices(expr: sp.Basic, fully_symmetric: set[sp.IndexedBase]) -> sp.Basic:
    
    def sort_indexed(i: sp.Indexed):
        base, indices = i.base, i.indices
        if base in fully_symmetric:
            sorted_indices = tuple(sorted(indices, key=sp.default_sort_key))
            return base[sorted_indices]
        return i

    return expr.replace(lambda x: isinstance(x, sp.Indexed), sort_indexed)

In [10]:
sort_indices(Z, {K, V})

\sqrt{|2\pi K|} + Sum(-\sqrt{|2\pi K|}*epsilon*K[rho1, rho2]*K[rho3, rho4]*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + Sum(-\sqrt{|2\pi K|}*epsilon*K[rho1, rho3]*K[rho2, rho4]*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + Sum(-\sqrt{|2\pi K|}*epsilon*K[rho1, rho4]*K[rho2, rho3]*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N))

# Two point correlator

$$
\mathbb{E}[z_{\mu_1} z_{\mu_2}] 
= \frac{1}{Z} \int \left[ \prod_{\mu} dz_\mu \right] 
e^{-S(z)} z_{\mu_1} z_{\mu_2} \\
= \frac{\sqrt{2\pi K}}{Z} \left\langle 
z_{\mu_1} z_{\mu_2} 
\exp\left( 
    -\frac{\epsilon}{24} 
    \sum_{\rho_1, \ldots, \rho_4} 
    V^{\rho_1 \rho_2 \rho_3 \rho_4} 
    z_{\rho_1} z_{\rho_2} z_{\rho_3} z_{\rho_4}
\right) 
\right\rangle_K
$$


In [11]:
inner = z[mu[0]]*z[mu[1]]*sp.exp(quartic_action)
limit = sp.series(inner, x=eps, x0=0, n=2).removeO()
limit = pull_sums_out_front(limit)
Ezz = Z0/Z * EK(limit)
Ezz = sp.simplify(Ezz)
Ezz

(epsilon*Sum(⟨z[mu1]*z[mu2]*z[rho1]*z[rho2]*z[rho3]*z[rho4]⟩_K*V[rho1, rho2, rho3, rho4], (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) - 24*⟨z[mu1]*z[mu2]⟩_K)/(epsilon*Sum(K[rho1, rho2]*K[rho3, rho4]*V[rho1, rho2, rho3, rho4], (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + epsilon*Sum(K[rho1, rho3]*K[rho2, rho4]*V[rho1, rho2, rho3, rho4], (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + epsilon*Sum(K[rho1, rho4]*K[rho2, rho3]*V[rho1, rho2, rho3, rho4], (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) - 24)

In [12]:
Ezz = sp.series(Ezz, x=eps, x0=0, n=2).removeO()
Ezz = wick_contraction(Ezz)
Ezz = wild_subs(Ezz, gaussian_rule)
Ezz

epsilon*(-24*(-Sum(K[rho1, rho2]*K[rho3, rho4]*V[rho1, rho2, rho3, rho4], (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N))/576 - Sum(K[rho1, rho3]*K[rho2, rho4]*V[rho1, rho2, rho3, rho4], (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N))/576 - Sum(K[rho1, rho4]*K[rho2, rho3]*V[rho1, rho2, rho3, rho4], (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N))/576)*K[mu1, mu2] - Sum((K[mu1, mu2]*K[rho1, rho2]*K[rho3, rho4] + K[mu1, mu2]*K[rho1, rho3]*K[rho2, rho4] + K[mu1, mu2]*K[rho1, rho4]*K[rho2, rho3] + K[mu1, rho1]*K[mu2, rho2]*K[rho3, rho4] + K[mu1, rho1]*K[mu2, rho3]*K[rho2, rho4] + K[mu1, rho1]*K[mu2, rho4]*K[rho2, rho3] + K[mu1, rho2]*K[mu2, rho1]*K[rho3, rho4] + K[mu1, rho2]*K[mu2, rho3]*K[rho1, rho4] + K[mu1, rho2]*K[mu2, rho4]*K[rho1, rho3] + K[mu1, rho3]*K[mu2, rho1]*K[rho2, rho4] + K[mu1, rho3]*K[mu2, rho2]*K[rho1, rho4] + K[mu1, rho3]*K[mu2, rho4]*K[rho1, rho2] + K[mu1, rho4]*K[mu2, rho1]*K[rho2, rho3] + K[mu1, rho4]*K[mu2, rho2]*K[rho1, rho3] + K[mu1, rho4]*K[m

In [13]:
Ezz = pull_sums_out_front(sp.expand(sp.simplify(Ezz)))
Ezz

K[mu1, mu2] + Sum(-epsilon*K[mu1, rho1]*K[mu2, rho2]*K[rho3, rho4]*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + Sum(-epsilon*K[mu1, rho1]*K[mu2, rho3]*K[rho2, rho4]*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + Sum(-epsilon*K[mu1, rho1]*K[mu2, rho4]*K[rho2, rho3]*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + Sum(-epsilon*K[mu1, rho2]*K[mu2, rho1]*K[rho3, rho4]*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + Sum(-epsilon*K[mu1, rho2]*K[mu2, rho3]*K[rho1, rho4]*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + Sum(-epsilon*K[mu1, rho2]*K[mu2, rho4]*K[rho1, rho3]*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) + Sum(-epsilon*K[mu1, rho3]*K[mu2, rho1]*K[rho2, rho4]*V[rho1, rho2, rho3, rho4]/24, (rho1, 1, N), (rho2, 1, N), (rho3, 1, N), (rho4, 1, N)) +